In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
df_reviews = pd.read_csv('beer_reviews.csv')
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   brewery_id          1586614 non-null  int64  
 1   brewery_name        1586599 non-null  object 
 2   review_time         1586614 non-null  int64  
 3   review_overall      1586614 non-null  float64
 4   review_aroma        1586614 non-null  float64
 5   review_appearance   1586614 non-null  float64
 6   review_profilename  1586266 non-null  object 
 7   beer_style          1586614 non-null  object 
 8   review_palate       1586614 non-null  float64
 9   review_taste        1586614 non-null  float64
 10  beer_name           1586614 non-null  object 
 11  beer_abv            1518829 non-null  float64
 12  beer_beerid         1586614 non-null  int64  
dtypes: float64(6), int64(3), object(4)
memory usage: 157.4+ MB


In [3]:
df_reviews.head(10)

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883
5,1075,Caldera Brewing Company,1325524659,3.0,3.5,3.5,oline73,Herbed / Spiced Beer,3.0,3.5,Caldera Ginger Beer,4.7,52159
6,1075,Caldera Brewing Company,1318991115,3.5,3.5,3.5,Reidrover,Herbed / Spiced Beer,4.0,4.0,Caldera Ginger Beer,4.7,52159
7,1075,Caldera Brewing Company,1306276018,3.0,2.5,3.5,alpinebryant,Herbed / Spiced Beer,2.0,3.5,Caldera Ginger Beer,4.7,52159
8,1075,Caldera Brewing Company,1290454503,4.0,3.0,3.5,LordAdmNelson,Herbed / Spiced Beer,3.5,4.0,Caldera Ginger Beer,4.7,52159
9,1075,Caldera Brewing Company,1285632924,4.5,3.5,5.0,augustgarage,Herbed / Spiced Beer,4.0,4.0,Caldera Ginger Beer,4.7,52159


In [4]:
# Group df_reviews by brewery name and beer name, then calculate the mean of the review scores for each group
df_scores = df_reviews.drop(['brewery_id', 'review_time', 'review_profilename', 'beer_style', 'beer_beerid', 'beer_abv'], axis=1)\
                     .groupby(by=['brewery_name', 'beer_name'], as_index=False).mean()

# Count the number of reviews for each brewery and beer combination
df_scores_count = df_reviews.drop(['brewery_id', 'review_time', 'review_profilename', 'beer_style', 'beer_beerid', 'beer_abv'], axis=1)\
                            .groupby(by=['brewery_name', 'beer_name'], as_index=False)['review_overall']\
                            .count()\
                            .rename(columns={'review_overall': 'number_of_reviews'})

# Merge the two DataFrames on brewery and beer names
df_scores = pd.merge(df_scores, df_scores_count, on=['brewery_name', 'beer_name'])

df_scores.head()

,brewery_name,beer_name,review_overall,review_aroma,review_appearance,review_palate,review_taste,number_of_reviews
0,'t Hofbrouwerijke,Blondelle,4.000000,4.500000,3.500000,4.000000,4.000000,1
1,'t Hofbrouwerijke,Bosprotter,3.722222,3.833333,3.666667,3.611111,3.777778,9
2,'t Hofbrouwerijke,Hof Korvatunturi,3.750000,4.000000,3.750000,3.500000,3.750000,2
3,'t Hofbrouwerijke,Hofblues,3.812500,3.812500,4.000000,3.625000,3.593750,16
4,'t Hofbrouwerijke,Hofdraak,3.357143,3.428571,3.714286,3.142857,3.428571,7


In [5]:
# Load second data set - Beer Profile
df_profile = pd.read_csv('beer_data_set.csv')
df_profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5558 entries, 0 to 5557
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         5556 non-null   object 
 1   key          5558 non-null   int64  
 2   Style        5558 non-null   object 
 3   Style Key    5558 non-null   int64  
 4   Brewery      5558 non-null   object 
 5   Description  5558 non-null   object 
 6   ABV          5558 non-null   float64
 7   Ave Rating   5558 non-null   float64
 8   Min IBU      5558 non-null   int64  
 9   Max IBU      5558 non-null   int64  
 10  Astringency  5558 non-null   int64  
 11  Body         5558 non-null   int64  
 12  Alcohol      5558 non-null   int64  
 13  Bitter       5558 non-null   int64  
 14  Sweet        5558 non-null   int64  
 15  Sour         5558 non-null   int64  
 16  Salty        5558 non-null   int64  
 17  Fruits       5558 non-null   int64  
 18  Hoppy        5558 non-null   int64  
 19  Spices

In [6]:
df_profile.head()

,Name,key,Style,Style Key,Brewery,Description,ABV,Ave Rating,Min IBU,Max IBU,...,Body,Alcohol,Bitter,Sweet,Sour,Salty,Fruits,Hoppy,Spices,Malty
0,Amber,251,Altbier,8,Alaskan Brewing Co.,"Notes:Richly malty and long on the palate, wit...",5.3,3.65,25,50,...,32,9,47,74,33,0,33,57,8,111
1,Double Bag,252,Altbier,8,Long Trail Brewing Co.,"Notes:This malty, full-bodied double alt is al...",7.2,3.90,25,50,...,57,18,33,55,16,0,24,35,12,84
2,Long Trail Ale,253,Altbier,8,Long Trail Brewing Co.,Notes:Long Trail Ale is a full-bodied amber al...,5.0,3.58,25,50,...,37,6,42,43,11,0,10,54,4,62
3,Doppelsticke,254,Altbier,8,Uerige Obergärige Hausbrauerei,Notes:,8.5,4.15,25,50,...,55,31,47,101,18,1,49,40,16,119
4,Scurry,255,Altbier,8,Off Color Brewing,Notes:Just cause it's dark and German doesn't ...,5.3,3.67,25,50,...,69,10,63,120,14,0,19,36,15,218


In [7]:
# Display `df_profile` rows with null values (missing beer names)
df_profile[df_profile['Name'].isnull()]

,Name,key,Style,Style Key,Brewery,Description,ABV,Ave Rating,Min IBU,Max IBU,...,Body,Alcohol,Bitter,Sweet,Sour,Salty,Fruits,Hoppy,Spices,Malty
1803,NaN,3504,Kvass,81,Monastyrskiy Kvas,Notes:,1.5,3.07,0,0,...,34,4,15,84,16,1,33,14,7,68
2150,NaN,2401,Lager - European Pale,57,Stella Artois,Notes:,5.0,3.11,18,25,...,14,10,20,19,3,0,1,26,3,26


In [8]:
# Look up data on 'Monastyrskiy Kvas' from `df_scores`
df_scores[df_scores['brewery_name']=='Monastyrskiy Kvas']


# Highlight important row
df_scores[df_scores['brewery_name']=='Monastyrskiy Kvas'].style.apply(
    lambda x: ['background: lightgreen' if x.name == 40781 else '' for i in x], 
    axis=1)

,brewery_name,beer_name,review_overall,review_aroma,review_appearance,review_palate,review_taste,number_of_reviews
40779,Monastyrskiy Kvas,Monastyrskiy Black Currant Kvas,3.500000,4.100000,3.600000,3.100000,3.600000,5
40780,Monastyrskiy Kvas,Monastyrskiy Cranberry Kvas,3.500000,3.833333,3.500000,3.000000,3.500000,3
40781,Monastyrskiy Kvas,Monastyrskiy Kvas,3.062500,3.187500,2.937500,2.812500,3.062500,8
40782,Monastyrskiy Kvas,Monastyrskiy Okroshka Kvas,3.625000,3.500000,3.250000,2.875000,3.625000,4
40783,Monastyrskiy Kvas,Verkin Kvas,3.125000,3.500000,3.125000,3.000000,2.750000,4
40784,Monastyrskiy Kvas,Verkin Pohmilny Kvas,3.500000,3.500000,2.500000,3.000000,3.500000,1


In [9]:
# Look up data on 'Stella Artois' from `df_scores`
df_scores[df_scores['brewery_name']=='Stella Artois']


# Highlight important row
df_scores[df_scores['brewery_name']=='Stella Artois'].style.apply(
    lambda x: ['background: lightgreen' if x.name == 55651 else '' for i in x], 
    axis=1)

,brewery_name,beer_name,review_overall,review_aroma,review_appearance,review_palate,review_taste,number_of_reviews
55635,Stella Artois,Artois Bock,3.277778,3.222222,3.555556,3.111111,3.111111,9
55636,Stella Artois,Atlas,1.857143,2.357143,2.714286,2.000000,1.928571,7
55637,Stella Artois,Bergenbrau,3.500000,3.000000,2.750000,3.250000,4.000000,2
55638,Stella Artois,Club De Stella Artois,3.384615,2.826923,3.240385,3.221154,3.134615,52
55639,Stella Artois,Dutch Gold Imported,2.833333,2.000000,2.833333,2.500000,2.166667,3
55640,Stella Artois,Eiken Artois,2.875000,2.625000,3.125000,2.500000,2.500000,4
55641,Stella Artois,Ginder Ale,3.666667,3.500000,3.666667,3.500000,3.333333,3
55642,Stella Artois,Horse-Ale,4.000000,4.166667,4.000000,4.333333,3.833333,3
55643,Stella Artois,La Becasse Framboise,2.666667,2.916667,3.416667,3.000000,2.666667,6
55644,Stella Artois,La Becasse Gueuze,3.166667,3.000000,3.000000,3.166667,2.500000,3


In [10]:
# Edit `Name` column for `Stella Artois` and `Monastyrskiy Kvas` on `df_profile`
df_profile.loc[[1803, 2150],'Name'] = df_profile.loc[[1803, 2150],'Brewery']

In [11]:
# Create new column on `df_profile` indicating whether the brewery name exists on `df_scores`
df_profile['brewery_review_exists'] = np.where(df_profile['Brewery'].isin(list(df_scores['brewery_name'].unique())), 1, 0)

# Formatting for better display
slice_ = 'brewery_review_exists'

def highlight_indicator(val):
    pink = 'background-color: pink' if val < 1 else ''
    return pink

df_profile.head(10).style.set_properties(**{'background-color': '#ffffb3'}, subset=slice_)\
                         .applymap(highlight_indicator, subset=[slice_])

,Name,key,Style,Style Key,Brewery,Description,ABV,Ave Rating,Min IBU,Max IBU,Astringency,Body,Alcohol,Bitter,Sweet,Sour,Salty,Fruits,Hoppy,Spices,Malty,brewery_review_exists
0,Amber,251,Altbier,8,Alaskan Brewing Co.,"Notes:Richly malty and long on the palate, with just enough hop backing to make this beautiful amber colored ""alt"" style beer notably well balanced.",5.300000,3.650000,25,50,13,32,9,47,74,33,0,33,57,8,111,1
1,Double Bag,252,Altbier,8,Long Trail Brewing Co.,"Notes:This malty, full-bodied double alt is also known as “Stickebier” – German slang for “secret brew”. Long Trail Double Bag was originally offered only in our brewery taproom as a special treat to our visitors. With an alcohol content of 7.2%, please indulge in moderation. The Long Trail Brewing Company is proud to have Double Bag named Malt Advocate’s “Beer of the Year” in 2001. Malt Advocate is a national magazine devoted to “expanding the boundaries of fine drinks”. Their panel of judges likes to keep things simple, and therefore of thousands of eligible competitors they award only two categories: “Imported” and “Domestic”. It is a great honor to receive this recognition.33 IBU",7.200000,3.900000,25,50,12,57,18,33,55,16,0,24,35,12,84,1
2,Long Trail Ale,253,Altbier,8,Long Trail Brewing Co.,"Notes:Long Trail Ale is a full-bodied amber ale modeled after the “Alt-biers” of Düsseldorf, Germany. Our top fermenting yeast and cold finishing temperature result in a complex, yet clean, full flavor. Originally introduced in November of 1989, Long Trail Ale beer quickly became, and remains, the largest selling craft-brew in Vermont. It is a multiple medal winner at the Great American Beer Festival.25 IBU",5.000000,3.580000,25,50,14,37,6,42,43,11,0,10,54,4,62,1
3,Doppelsticke,254,Altbier,8,Uerige Obergärige Hausbrauerei,Notes:,8.500000,4.150000,25,50,13,55,31,47,101,18,1,49,40,16,119,0
4,Scurry,255,Altbier,8,Off Color Brewing,"Notes:Just cause it's dark and German doesn't mean it's an alt. Based on the obscure Kottbusser style of beer, this surprisingly dry beer retains all the aroma and nuance of honey (from bees) and molasses (not from bees) creating a perception of sweetness through cool, controlled fermentation. Because we're sweet enough damn it. 18 IBU",5.300000,3.670000,25,50,21,69,10,63,120,14,0,19,36,15,218,0
5,Sleigh'r Dark Doüble Alt Ale,256,Altbier,8,Ninkasi Brewing Company,"Notes:Called 'Dark Double Alt' on the label.Seize the season with Sleigh'r. Layers of deeply toasted malt are balanced by just enough hop bitterness to make it deceivingly drinkable. Paired with a dry finish, Sleigh’r is anything but your typical winter brew.An Alt ferments with Ale yeast at colder lagering temperatures. This effect gives Alts a more refined, crisp lager-like flavor than traditional ales. The Alt has been “Ninkasified” raising the ABV and IBUs. Sleigh'r has a deep, toasted malt flavor that finishes dry and balanced.50 IBU",7.200000,3.780000,25,50,25,51,26,44,45,9,1,11,51,20,95,1
6,Sticke,257,Altbier,8,Uerige Obergärige Hausbrauerei,Notes:,6.000000,4.100000,25,50,22,45,13,46,62,25,1,34,60,4,103,0
7,Okto Festival Ale,258,Altbier,8,Widmer Brothers Brewing Company,Notes:Classified as a German Altbier by the Widmer Brothers; brewed with an (ale) Alt yeast strain.availability: early Aug. - late Oct.bitterness : 28 IBUoriginal gravity: 13.5° PLATO,5.300000,3.460000,25,50,28,40,3,40,58,29,0,36,54,8,97,1
8,Southampton Altbier,259,Altbier,8,Southampton Publick House,Notes:,5.000000,3.600000,25,50,18,49,5,37,73,22,0,21,37,4,98,1
9,Copper,260,Altbier,8,The Olde Mecklenburg Brewery,"Notes:OMB’s flagship, Copper, is an authentic Düsseldorf style “Altbier” or Amber Ale. There are no crazy flavors or gimmicks with Copper – its full-flavored start transitions to a smooth, balanced, easy-drinking finish. It’s the beer that tastes like beer.",4.800000,4.100000,25,50,25,35,4,38,39,13,1,8,60,16,97,1


In [12]:
# Create new data frame (`brewery_no_scores`) listing breweries on `df_profile` with no exact match on `df_scores`
brewery_no_scores = pd.DataFrame(df_profile[df_profile['brewery_review_exists']==0]['Brewery'].unique()).set_axis(['Brewery'], axis=1)
brewery_no_scores

,Brewery
0,Uerige Obergärige Hausbrauerei
1,Off Color Brewing
2,Victory Brewing Company - Downingtown
3,Hops & Grain Brewing
4,Union Craft Brewing Company
...,...
538,Dust Bowl Brewing Company - Downtown Taproom
539,Griffin Claw Brewing Company
540,Anthem Brewing Company
541,Aslin Beer Company


In [13]:
# Create new column on `df_scores` indicating whether the brewery name exists on `df_profile`
df_scores['brewery_profile_exists'] = np.where(df_scores['brewery_name'].isin(list(df_profile['Brewery'].unique())), 1, 0)

# Create new data frame (`brewery_no_profile`) listing breweries on `df_scores` with no exact match on `df_profile`
brewery_no_profile = pd.DataFrame(df_scores[df_scores['brewery_profile_exists']==0]['brewery_name'].unique()).set_axis(['Brewery'], axis=1)

In [14]:
# Load table containing saved brewery name matches ("Brewery Name Fuzzy Match List.csv")
fuzzy_match_brewery = pd.read_csv("Brewery Name Fuzzy Match List.csv")
fuzzy_match_brewery

,Brewery,matches
0,Žatecký Pivovar,Žatecký Pivovar
1,Abbey Brewing Co. (Monastery Of Christ In The ...,Abbey Brewing Co.
2,Aktienbrauerei Kaufbeuren AG,Aktien-Brauerei Kaufbeuren AG
3,Alltech's Lexington Brewing and Distilling Co.,Alltech’s Lexington Brewing and Distilling Co.
4,Asia Pacific Breweries Limited,Asia Pacific Breweries Ltd.
...,...,...
82,Urban Chestnut Brewing Company - Midtown Brewe...,Urban Chestnut Brewing Company
83,Victory Brewing Company - Downingtown,Victory Brewing Company
84,Volfas Engelman,Vofas-Engelman
85,Water Street Brewing Co.,Water Street Brewing & Ale House


In [15]:
# Save table as dictionary
fuzzy_match_dict = dict(zip(fuzzy_match_brewery['Brewery'], fuzzy_match_brewery['matches']))

# Replace relevant brewery names in `df_profile`
df_profile['Brewery'] = df_profile['Brewery'].replace(fuzzy_match_dict)

In [16]:
# Correcting 'brewery_review_exists' indicator column values on `df_profile` after using fuzzy matches
df_profile['brewery_review_exists'] = np.where(df_profile['Brewery'].isin(list(df_scores['brewery_name'].unique())), 1, 0)

# Correcting 'brewery_profile_exists' indicator column values on `df_scores` after using fuzzy matches
df_scores['brewery_profile_exists'] = np.where(df_scores['brewery_name'].isin(list(df_profile['Brewery'].unique())), 1, 0)

In [17]:
# Remove all brews from breweries that are not listed in either `df_profile` or `df_scores`
df_scores.drop(df_scores[df_scores['brewery_profile_exists']==0].index, axis=0, inplace=True)
df_profile.drop(df_profile[df_profile['brewery_review_exists']==0].index, axis=0, inplace=True)

In [18]:
# Create new identifier column in `df_scores` called 'beer_name_full'
# Fill column by concatenating `brewery_name` and `beer_name`
df_scores['beer_name_full'] = df_scores['brewery_name'] + ' ' + df_scores['beer_name']

# Check if all generated brew names in df_scores are unique:
df_scores['beer_name_full'].nunique() == len(df_scores.index)

True

In [19]:
# Create new identifier column in `df_profile` called 'Beer Name (Full)' 
# Fill column by concatenating `Brewery` and `Name`
df_profile['Beer Name (Full)'] = df_profile['Brewery'] + ' ' + df_profile['Name']

# Check if all generated brew names in `df_profile` are unique:
df_profile['Beer Name (Full)'].nunique() == len(df_profile.index)

False

In [20]:
# Check duplicated brew name
df_profile[df_profile['Beer Name (Full)'].duplicated()]

,Name,key,Style,Style Key,Brewery,Description,ABV,Ave Rating,Min IBU,Max IBU,...,Bitter,Sweet,Sour,Salty,Fruits,Hoppy,Spices,Malty,brewery_review_exists,Beer Name (Full)
4056,Crazy Jackass Ale,594,Rye Beer - Roggenbier,15,Sweetwater Tavern & Brewery,Notes:,5.5,3.84,10,20,...,0,0,0,0,0,0,0,0,1,Sweetwater Tavern & Brewery Crazy Jackass Ale


In [21]:
# List rows with duplicated brew name
df_profile[df_profile['Beer Name (Full)']=='Sweetwater Tavern & Brewery Crazy Jackass Ale']


# Highlight row with error
df_profile[df_profile['Beer Name (Full)']=='Sweetwater Tavern & Brewery Crazy Jackass Ale'].style.apply(
    lambda x: ['background: pink' if x.name == 4056 else '' for i in x], 
    axis=1)

,Name,key,Style,Style Key,Brewery,Description,ABV,Ave Rating,Min IBU,Max IBU,Astringency,Body,Alcohol,Bitter,Sweet,Sour,Salty,Fruits,Hoppy,Spices,Malty,brewery_review_exists,Beer Name (Full)
4030,Crazy Jackass Ale,568,Rye Beer - Roggenbier,15,Sweetwater Tavern & Brewery,Notes:,6.000000,3.960000,10,20,2,5,1,6,4,3,0,5,8,4,26,1,Sweetwater Tavern & Brewery Crazy Jackass Ale
4056,Crazy Jackass Ale,594,Rye Beer - Roggenbier,15,Sweetwater Tavern & Brewery,Notes:,5.500000,3.840000,10,20,0,0,0,0,0,0,0,0,0,0,0,1,Sweetwater Tavern & Brewery Crazy Jackass Ale


In [22]:
# Remove duplicate row containing error (missing data)
df_profile.drop(4056, inplace=True)

In [23]:
# Create new column on `df_profile` indicating whether the complete brew name exists on `df_scores`
df_profile['beer_review_exists'] = np.where(df_profile['Beer Name (Full)'].isin(list(df_scores['beer_name_full'])), 1, 0)


# Formatting for better display
slice_ = 'beer_review_exists'

df_profile.head(10).style.set_properties(**{'background-color': '#ffffb3'}, subset=slice_)\
                         .applymap(highlight_indicator, subset=[slice_])

,Name,key,Style,Style Key,Brewery,Description,ABV,Ave Rating,Min IBU,Max IBU,Astringency,Body,Alcohol,Bitter,Sweet,Sour,Salty,Fruits,Hoppy,Spices,Malty,brewery_review_exists,Beer Name (Full),beer_review_exists
0,Amber,251,Altbier,8,Alaskan Brewing Co.,"Notes:Richly malty and long on the palate, with just enough hop backing to make this beautiful amber colored ""alt"" style beer notably well balanced.",5.300000,3.650000,25,50,13,32,9,47,74,33,0,33,57,8,111,1,Alaskan Brewing Co. Amber,0
1,Double Bag,252,Altbier,8,Long Trail Brewing Co.,"Notes:This malty, full-bodied double alt is also known as “Stickebier” – German slang for “secret brew”. Long Trail Double Bag was originally offered only in our brewery taproom as a special treat to our visitors. With an alcohol content of 7.2%, please indulge in moderation. The Long Trail Brewing Company is proud to have Double Bag named Malt Advocate’s “Beer of the Year” in 2001. Malt Advocate is a national magazine devoted to “expanding the boundaries of fine drinks”. Their panel of judges likes to keep things simple, and therefore of thousands of eligible competitors they award only two categories: “Imported” and “Domestic”. It is a great honor to receive this recognition.33 IBU",7.200000,3.900000,25,50,12,57,18,33,55,16,0,24,35,12,84,1,Long Trail Brewing Co. Double Bag,1
2,Long Trail Ale,253,Altbier,8,Long Trail Brewing Co.,"Notes:Long Trail Ale is a full-bodied amber ale modeled after the “Alt-biers” of Düsseldorf, Germany. Our top fermenting yeast and cold finishing temperature result in a complex, yet clean, full flavor. Originally introduced in November of 1989, Long Trail Ale beer quickly became, and remains, the largest selling craft-brew in Vermont. It is a multiple medal winner at the Great American Beer Festival.25 IBU",5.000000,3.580000,25,50,14,37,6,42,43,11,0,10,54,4,62,1,Long Trail Brewing Co. Long Trail Ale,1
3,Doppelsticke,254,Altbier,8,Uerige Obergärige Hausbrauerei GmbH / Zum Uerige,Notes:,8.500000,4.150000,25,50,13,55,31,47,101,18,1,49,40,16,119,1,Uerige Obergärige Hausbrauerei GmbH / Zum Uerige Doppelsticke,0
5,Sleigh'r Dark Doüble Alt Ale,256,Altbier,8,Ninkasi Brewing Company,"Notes:Called 'Dark Double Alt' on the label.Seize the season with Sleigh'r. Layers of deeply toasted malt are balanced by just enough hop bitterness to make it deceivingly drinkable. Paired with a dry finish, Sleigh’r is anything but your typical winter brew.An Alt ferments with Ale yeast at colder lagering temperatures. This effect gives Alts a more refined, crisp lager-like flavor than traditional ales. The Alt has been “Ninkasified” raising the ABV and IBUs. Sleigh'r has a deep, toasted malt flavor that finishes dry and balanced.50 IBU",7.200000,3.780000,25,50,25,51,26,44,45,9,1,11,51,20,95,1,Ninkasi Brewing Company Sleigh'r Dark Doüble Alt Ale,1
6,Sticke,257,Altbier,8,Uerige Obergärige Hausbrauerei GmbH / Zum Uerige,Notes:,6.000000,4.100000,25,50,22,45,13,46,62,25,1,34,60,4,103,1,Uerige Obergärige Hausbrauerei GmbH / Zum Uerige Sticke,0
7,Okto Festival Ale,258,Altbier,8,Widmer Brothers Brewing Company,Notes:Classified as a German Altbier by the Widmer Brothers; brewed with an (ale) Alt yeast strain.availability: early Aug. - late Oct.bitterness : 28 IBUoriginal gravity: 13.5° PLATO,5.300000,3.460000,25,50,28,40,3,40,58,29,0,36,54,8,97,1,Widmer Brothers Brewing Company Okto Festival Ale,0
8,Southampton Altbier,259,Altbier,8,Southampton Publick House,Notes:,5.000000,3.600000,25,50,18,49,5,37,73,22,0,21,37,4,98,1,Southampton Publick House Southampton Altbier,1
9,Copper,260,Altbier,8,The Olde Mecklenburg Brewery,"Notes:OMB’s flagship, Copper, is an authentic Düsseldorf style “Altbier” or Amber Ale. There are no crazy flavors or gimmicks with Copper – its full-flavored start transitions to a smooth, balanced, easy-drinking finish. It’s the beer that tastes like beer.",4.800000,4.100000,25,50,25,35,4,38,39,13,1,8,60,16,97,1,The Olde Mecklenburg Brewery Copper,0
10,Organic Münster Alt,261,Altbier,8,Br

In [24]:
# Create new data frame (`beer_no_scores`) listing brews on `df_profile` with no exact match on `df_scores`
beer_no_scores = pd.DataFrame(df_profile[df_profile['beer_review_exists']==0]['Beer Name (Full)']).set_axis(['Beer Name (Full)'], axis=1)
beer_no_scores

,Beer Name (Full)
0,Alaskan Brewing Co. Amber
3,Uerige Obergärige Hausbrauerei GmbH / Zum Ueri...
6,Uerige Obergärige Hausbrauerei GmbH / Zum Ueri...
7,Widmer Brothers Brewing Company Okto Festival Ale
9,The Olde Mecklenburg Brewery Copper
...,...
5551,Fremont Brewing Company Winter Ale
5552,RJ Rockers Brewing Company The First Snow Ale
5554,Anchor Brewing Company Our Special Ale 2019 (A...
5556,"Shmaltz Brewing Company He'Brew Hanukkah, Chan..."


In [25]:
# Create new column on `df_scores` indicating whether the complete brew name exists on `df_profile`
df_scores['beer_profile_exists'] = np.where(df_scores['beer_name_full'].isin(list(df_profile['Beer Name (Full)'])), 1, 0)

# Create new data frame (`beer_no_profile`) listing brews on `df_scores` with no exact match on `df_profile`
beer_no_profile = pd.DataFrame(df_scores[df_scores['beer_profile_exists']==0]['beer_name_full']).set_axis(['beer_name_full'], axis=1)

In [26]:
fuzzy_match_beer_name = pd.read_csv('Beer Name Fuzzy Match List.csv')
fuzzy_match_beer_name

,Beer Name (Full),matches
0,(512) Brewing Company (512) Pecan Porter(512) ...,(512) Brewing Company (512) Pecan Porter
1,(512) Brewing Company (512) Whiskey Barrel Age...,(512) Brewing Company (512) Whiskey Barrel Age...
2,Žatecký Pivovar Žatec Dark Lager,Žatecký Pivovar Žatec Dark Lager
3,21st Amendment Brewery Brew Free Or Die IPA,21st Amendment Brewery 21st Amendment IPA
4,21st Amendment Brewery Lower De Boom Barleywine,21st Amendment Brewery Lower Da Boom Barleywine
...,...,...
1083,Wychwood Brewery Company Ltd Bah Humbug! Chris...,Wychwood Brewery Company Ltd Bah Humbug
1084,Wychwood Brewery Company Ltd Hobgoblin Ruby,Wychwood Brewery Company Ltd Hobgoblin
1085,Yards Brewing Co. Washington's Porter,Yards Brewing Co. General Washington's Tavern ...
1086,Zywiec Breweries PLC (Heineken) Tatra MocneZyw...,Zywiec Breweries PLC (Heineken) Tatra


In [27]:
# Save table as dictionary
fuzzy_match_dict = dict(zip(fuzzy_match_beer_name['Beer Name (Full)'], fuzzy_match_beer_name['matches']))

# Replace relevant brewery names in `df_profile`
df_profile['Beer Name (Full)'] = df_profile['Beer Name (Full)'].replace(fuzzy_match_dict)

In [28]:
# Correcting 'beer_review_exist' indicator column values on `df_profile` after using fuzzy matches
df_profile['beer_review_exists'] = np.where(df_profile['Beer Name (Full)'].isin(list(df_scores['beer_name_full'].unique())), 1, 0)

# Correcting 'beer_profile_exists' indicator column values on `df_scores` after using fuzzy matches
df_scores['beer_profile_exists'] = np.where(df_scores['beer_name_full'].isin(list(df_profile['Beer Name (Full)'].unique())), 1, 0)

In [29]:
# Remove all brews that are not listed in either `df_profile` or `df_scores`
df_scores.drop(df_scores[df_scores['beer_profile_exists']==0].index, axis=0, inplace=True)
df_profile.drop(df_profile[df_profile['beer_review_exists']==0].index, axis=0, inplace=True)

In [30]:
len(df_scores.index)

3188

In [31]:
len(df_profile.index)

3188

In [32]:
# Drop columns that are redundant or no longer neccessary
df_profile.drop(['key', 'Style Key', 'brewery_review_exists', 'beer_review_exists', 'Ave Rating'], axis=1, inplace=True)
df_scores.drop(['brewery_name', 'beer_name', 'brewery_profile_exists', 'beer_profile_exists'], axis=1, inplace=True)


df_scores.rename(columns={"beer_name_full": "Beer Name (Full)"}, inplace=True)

In [33]:
# Join data frames to make new dataset
df_final = pd.merge(df_profile, df_scores, how='left', on=['Beer Name (Full)'])

# Re-arrange column order
df_final = df_final[['Name', 'Style', 'Brewery', 'Beer Name (Full)', 
                     'Description', 'ABV', 'Min IBU', 'Max IBU', 
                     'Astringency', 'Body', 'Alcohol', 
                     'Bitter', 'Sweet', 'Sour', 'Salty', 'Fruits', 'Hoppy', 'Spices', 'Malty', 
                     'review_aroma','review_appearance', 'review_palate', 'review_taste', 
                     'number_of_reviews', 'review_overall']]

In [34]:
# Resulting dataset:
df_final

,Name,Style,Brewery,Beer Name (Full),Description,ABV,Min IBU,Max IBU,Astringency,Body,...,Fruits,Hoppy,Spices,Malty,review_aroma,review_appearance,review_palate,review_taste,number_of_reviews,review_overall
0,Amber,Altbier,Alaskan Brewing Co.,Alaskan Brewing Co. Alaskan Amber,"Notes:Richly malty and long on the palate, wit...",5.3,25,50,13,32,...,33,57,8,111,3.498994,3.636821,3.556338,3.643863,497,3.847082
1,Double Bag,Altbier,Long Trail Brewing Co.,Long Trail Brewing Co. Double Bag,"Notes:This malty, full-bodied double alt is al...",7.2,25,50,12,57,...,24,35,12,84,3.798337,3.846154,3.904366,4.024948,481,4.034304
2,Long Trail Ale,Altbier,Long Trail Brewing Co.,Long Trail Brewing Co. Long Trail Ale,Notes:Long Trail Ale is a full-bodied amber al...,5.0,25,50,14,37,...,10,54,4,62,3.409814,3.667109,3.600796,3.631300,377,3.830239
3,Doppelsticke,Altbier,Uerige Obergärige Hausbrauerei GmbH / Zum Uerige,Uerige Obergärige Hausbrauerei GmbH / Zum Ueri...,Notes:,8.5,25,50,13,55,...,49,40,16,119,4.148098,4.033967,4.150815,4.205163,368,4.005435
4,Sleigh'r Dark Doüble Alt Ale,Altbier,Ninkasi Brewing Company,Ninkasi Brewing Company Sleigh'r Dark Doüble A...,Notes:Called 'Dark Double Alt' on the label.Se...,7.2,25,50,25,51,...,11,51,20,95,3.625000,3.973958,3.734375,3.765625,96,3.817708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3183,Winter Shredder,Winter Warmer,Cisco Brewers Inc.,Cisco Brewers Inc. Winter Shredder,Notes:,8.8,35,50,15,37,...,23,45,67,74,4.125000,3.875000,3.875000,3.750000,4,4.000000
3184,The First Snow Ale,Winter Warmer,RJ Rockers Brewing Company,RJ Rockers Brewing Company First Snow Ale,Notes:This hearty American pale ale contains a...,6.0,35,50,15,31,...,54,14,140,58,3.950000,3.750000,3.760000,3.790000,50,3.730000
3185,Red Nose Winter Ale,Winter Warmer,Natty Greene's Pub & Brewing Co.,Natty Greene's Pub & Brewing Co. Red Nose Wint...,Notes:,6.8,35,50,8,44,...,26,21,96,77,3.576923,3.711538,3.596154,3.673077,26,3.711538
3186,Fish Tale Winterfish,Winter Warmer,Fish Brewing Company / Fishbowl Brewpub,Fish Brewing Company / Fishbowl Brewpub Fish T...,Notes:,7.5,35,50,11,36,...,81,110,18,73,3.902299,3.885057,3.862069,3.902299,87,3.879310


In [35]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3188 entries, 0 to 3187
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               3188 non-null   object 
 1   Style              3188 non-null   object 
 2   Brewery            3188 non-null   object 
 3   Beer Name (Full)   3188 non-null   object 
 4   Description        3188 non-null   object 
 5   ABV                3188 non-null   float64
 6   Min IBU            3188 non-null   int64  
 7   Max IBU            3188 non-null   int64  
 8   Astringency        3188 non-null   int64  
 9   Body               3188 non-null   int64  
 10  Alcohol            3188 non-null   int64  
 11  Bitter             3188 non-null   int64  
 12  Sweet              3188 non-null   int64  
 13  Sour               3188 non-null   int64  
 14  Salty              3188 non-null   int64  
 15  Fruits             3188 non-null   int64  
 16  Hoppy              3188 

In [36]:
df_beer = df_final.drop(['Name', 'Brewery', 'Beer Name (Full)', 'Description'], axis=1)

In [37]:
df_beer.columns

Index(['Style', 'ABV', 'Min IBU', 'Max IBU', 'Astringency', 'Body', 'Alcohol',
       'Bitter', 'Sweet', 'Sour', 'Salty', 'Fruits', 'Hoppy', 'Spices',
       'Malty', 'review_aroma', 'review_appearance', 'review_palate',
       'review_taste', 'number_of_reviews', 'review_overall'],
      dtype='object')

In [38]:
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder object
le = LabelEncoder()

# Fit the LabelEncoder to the Style column and transform its values
df_beer['Style'] = le.fit_transform(df_beer['Style'])

In [39]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Define X (features) and y (target)
X = df_beer.drop('review_overall', axis=1)
y = df_beer['review_overall']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the linear regression model
model = LinearRegression()

# Fit the model to the training data
model.fit(X_train, y_train)

# Predict the target variable for the test data
y_pred = model.predict(X_test)

# Evaluate the model using mean squared error
mse = mean_squared_error(y_test, y_pred)
print('Mean squared error:', mse)

Mean squared error: 0.021663874401229733


In [40]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_beer.drop('review_overall', axis=1), df_beer['review_overall'], test_size=0.2, random_state=42)

# Instantiate a Random Forest regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model on the training data
rf.fit(X_train, y_train)

# Predict the review_overall values for the test data
y_pred = rf.predict(X_test)

# Evaluate the model using metrics such as mean squared error (MSE) and R-squared
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print('Mean squared error:', mse)
print('R-squared:', r2)

Mean squared error: 0.022950145780684963
R-squared: 0.888495994890612


In [41]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

# Define the model architecture
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=len(df_beer.columns)-1))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(loss='mean_squared_error', optimizer=Adam())

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
mse = model.evaluate(X_test, y_test)
print('MSE:', mse)

2023-04-11 16:38:28.853349: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-11 16:38:39.703177: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100
40/40 [==============================] - 1s 5ms/step - loss: 857.3907 - val_loss: 18.9652
Epoch 2/100
40/40 [==============================] - 0s 2ms/step - loss: 180.4851 - val_loss: 10.3605
Epoch 3/100
40/40 [==============================] - 0s 2ms/step - loss: 114.6700 - val_loss: 9.2015
Epoch 4/100
40/40 [==============================] - 0s 2ms/step - loss: 85.2725 - val_loss: 17.8319
Epoch 5/100
40/40 [==============================] - 0s 2ms/step - loss: 67.0664 - val_loss: 6.7368
Epoch 6/100
40/40 [==============================] - 0s 1ms/step - loss: 55.4547 - val_loss: 5.5937
Epoch 7/100
40/40 [==============================] - 0s 1ms/step - loss: 40.8889 - val_loss: 8.9191
Epoch 8/100
40/40 [==============================] - 0s 1ms/step - loss: 38.5085 - val_loss: 4.5910
Epoch 9/100
40/40 [==============================] - 0s 2ms/step - loss: 29.1828 - val_loss: 4.3164
Epoch 10/100
40/40 [==============================] - 0s 1ms/step - loss: 24.6184 - val_loss: 

Epoch 83/100
40/40 [==============================] - 0s 1ms/step - loss: 0.4581 - val_loss: 0.2248
Epoch 84/100
40/40 [==============================] - 0s 2ms/step - loss: 0.4183 - val_loss: 0.2191
Epoch 85/100
40/40 [==============================] - 0s 1ms/step - loss: 0.5236 - val_loss: 0.3856
Epoch 86/100
40/40 [==============================] - 0s 2ms/step - loss: 0.4964 - val_loss: 0.2034
Epoch 87/100
40/40 [==============================] - 0s 1ms/step - loss: 0.4062 - val_loss: 0.2149
Epoch 88/100
40/40 [==============================] - 0s 2ms/step - loss: 0.3990 - val_loss: 0.1930
Epoch 89/100
40/40 [==============================] - 0s 2ms/step - loss: 0.4081 - val_loss: 0.2226
Epoch 90/100
40/40 [==============================] - 0s 2ms/step - loss: 0.4350 - val_loss: 0.2029
Epoch 91/100
40/40 [==============================] - 0s 2ms/step - loss: 0.3630 - val_loss: 0.1840
Epoch 92/100
40/40 [==============================] - 0s 2ms/step - loss: 0.4279 - val_loss: 0.2195
